In [1]:
from pdf_rag import query_rag

In [6]:
import sys
print(sys.version)

3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


In [2]:
query_rag("what is the plans")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/pull "HTTP/1.1 200 OK"
c:\1_files\ojt\light_weight_solutions\langgraph_multi_agent_with_tools_and_rag_using_gpt_4o\pdf_rag.py:56: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:root:Loaded existing vector database.
INFO:root:Retriever created.
INFO:root:Chain created successfully.
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.multi_query:Generated queries: ['Here are five different versions of the original qu

'According to the provided context, NetFast offers three internet plans:\n\n1. Basic Plan - $29.99/month - Speed: 50 Mbps - Suitable for browsing and streaming\n2. Standard Plan - $49.99/month - Speed: 150 Mbps - Great for families and work-from-home\n3. Premium Plan - $79.99/month - Speed: 500 Mbps - Best for gaming and ultra-fast streaming'

In [3]:
import json
# Load the JSON file
file_path = "monthly_expenses.json"  # Replace with the correct path to your file
with open(file_path, "r") as file:
    data = json.load(file)

monthly_expenses_json_string = json.dumps(data, indent=4) 

In [4]:
from langchain_ollama import ChatOllama
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

#MODEL_NAME = "qwen2.5:7b-instruct"
#model = ChatOllama(model=MODEL_NAME, temperature=0.0)


load_dotenv()
model = ChatOpenAI(model="gpt-4o", temperature=0.7)
#model = ChatOpenAI(model="gpt-4o")
# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

def rag_search(query: str) -> str:
    """Search the information about netfast internet company"""

    rag_result = query_rag(query)
    return (rag_result)

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

report_agent = create_react_agent(
    model=model,
    tools=[],
    name="report_expert",
    prompt=f"You are a report expert. you can give reports based on this data:{monthly_expenses_json_string }"
)

netfast_agent = create_react_agent(
    model=model,
    tools=[rag_search],
    name="netfast_expert",
    prompt=f"You are a netfast expert. you always use the tool rag_search"
)

# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent, report_agent, netfast_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
        "For report queries, use report_agent."
        "For netfast queries, use netfast_agent."
    ),
    output_mode="full_history"
)



In [7]:
model 

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023EA65B0850>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023EB5486D40>, root_client=<openai.OpenAI object at 0x0000023EB546F7F0>, root_async_client=<openai.AsyncOpenAI object at 0x0000023EAFF2CC40>, model_name='gpt-4o', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [5]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "give me the summary of the report, what is the average spending each parameter each year"
        }
    ]
})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

give me the summary of the report, what is the average spending each parameter each year
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_report_expert (call_Gzrcp6RNIVDHc8HUEG6E9Thv)
 Call ID: call_Gzrcp6RNIVDHc8HUEG6E9Thv
  Args:
================================= Tool Message =================================
Name: transfer_to_report_expert

Successfully transferred to report_expert
================================== Ai Message ==================================
Name: report_expert

To calculate the average spending for each parameter (Food, Transpo, Daily) for each year, we can sum the monthly values for each parameter and then divide by the number of months (12) in each year. Here is the summary:

### 2023
- **Food**
  - Total: \(8000 + 7500 + 8500 + 8000 + 7800 + 8200 + 8500 + 8700 + 8100 + 8300 + 8600 + 9000 = 99200\)
  - Ave

In [10]:
app = workflow.compile()

# Collect all steps
data = 1
for step in app.stream({
    "messages": [
        {
            "role": "user",
            "content": "WHAT ARE THE AVAILABLE PLANS"
        }
    ]
}):
    print(step) 
    data = step       # Live output
    

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'supervisor': {'messages': [HumanMessage(content='WHAT ARE THE AVAILABLE PLANS', additional_kwargs={}, response_metadata={}, id='6ef34942-060a-470c-ac64-7214f935a901'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cmiEuruOXvk4aglJaZ3YIItN', 'function': {'arguments': '{}', 'name': 'transfer_to_netfast_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 162, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7faba9ef5', 'id': 'chatcmpl-BKyeYTOMwQQ8B4nd6v4V5ZC8oG86K', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-95d133e9-a705-48ae-b753-cab1082b1a72-0', tool_calls=[{'name': 'transfer_to_netfast_expert', 'args': {}, 'id': 'call

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/pull "HTTP/1.1 200 OK"
c:\1_files\ojt\light_weight_solutions\langgraph_multi_agent_with_tools_and_rag_using_gpt_4o\pdf_rag.py:56: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(
INFO:root:Loaded existing vector database.
INFO:root:Retriever created.
INFO:root:Chain created successfully.
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.multi_query:Generated queries: ['Here are five different versions of the original question:', '1. What are the current internet plans offered by N

{'netfast_expert': {'messages': [HumanMessage(content='WHAT ARE THE AVAILABLE PLANS', additional_kwargs={}, response_metadata={}, id='6ef34942-060a-470c-ac64-7214f935a901'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cmiEuruOXvk4aglJaZ3YIItN', 'function': {'arguments': '{}', 'name': 'transfer_to_netfast_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 162, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7faba9ef5', 'id': 'chatcmpl-BKyeYTOMwQQ8B4nd6v4V5ZC8oG86K', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-95d133e9-a705-48ae-b753-cab1082b1a72-0', tool_calls=[{'name': 'transfer_to_netfast_expert', 'args': {}, 'id': '

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'supervisor': {'messages': [HumanMessage(content='WHAT ARE THE AVAILABLE PLANS', additional_kwargs={}, response_metadata={}, id='6ef34942-060a-470c-ac64-7214f935a901'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cmiEuruOXvk4aglJaZ3YIItN', 'function': {'arguments': '{}', 'name': 'transfer_to_netfast_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 162, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7faba9ef5', 'id': 'chatcmpl-BKyeYTOMwQQ8B4nd6v4V5ZC8oG86K', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-95d133e9-a705-48ae-b753-cab1082b1a72-0', tool_calls=[{'name': 'transfer_to_netfast_expert', 'args': {}, 'id': 'call

In [11]:
for m in data['supervisor']['messages']:
    m.pretty_print()

================================ Human Message =================================

WHAT ARE THE AVAILABLE PLANS
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_netfast_expert (call_cmiEuruOXvk4aglJaZ3YIItN)
 Call ID: call_cmiEuruOXvk4aglJaZ3YIItN
  Args:
================================= Tool Message =================================
Name: transfer_to_netfast_expert

Successfully transferred to netfast_expert
================================== Ai Message ==================================
Name: netfast_expert
Tool Calls:
  rag_search (call_9abRsQHMYGhd5DqB1fx0ycPV)
 Call ID: call_9abRsQHMYGhd5DqB1fx0ycPV
  Args:
    query: available plans of Netfast internet company
================================= Tool Message =================================
Name: rag_search

According to the document, NetFast offers three internet plans:

1. Basic Plan - $29.99/month - Speed: 50 Mbps
2. Standard Plan - $49.99/month - Speed: